# Taro's STATSM148 Notebook

### I. Data Cleaning and Feature Engineering

In [2]:
import pandas as pd
import numpy as np

# Import initial data
data = pd.read_csv('export.csv')
event_defs = pd.read_csv('event_definitions.csv')

In [4]:
# Merge export.csv with event_definitions.csv
merged = pd.merge(data, event_defs, left_on='ed_id', right_on='event_definition_id', how='inner')

In [5]:
# Convert timestamps to datetime
merged['event_timestamp'] = pd.to_datetime(merged['event_timestamp'])

In [6]:
# Drop duplicate columns
merged = merged.drop(['event_name_x', 'event_definition_id'], axis=1)

In [7]:
# Notice how there are more account ids than customer ids
# Indicates that each customer can have multiple accounts
merged.customer_id.nunique()
merged.account_id.nunique()

1665623

In [10]:
# Organize dataset by customer
merged = merged.sort_values(by='customer_id')

In [11]:
# Add n_accounts column to represent the number of accounts each customer has
account_counts = merged.groupby('customer_id').size().reset_index(name='n_accounts')

merged_with_counts = pd.merge(merged, account_counts, on='customer_id')

In [12]:
# The distribution of n_accounts per customer (kinda crazy!)
merged_with_counts.groupby('customer_id')['n_accounts'].mean().describe()

count    1.665623e+06
mean     3.385285e+01
std      4.481940e+01
min      1.000000e+00
25%      8.000000e+00
50%      2.100000e+01
75%      4.200000e+01
max      3.277000e+03
Name: n_accounts, dtype: float64

In [13]:
# The customer with the most accounts info:
merged_with_counts.iloc[merged_with_counts['n_accounts'].idxmax(), ]

customer_id                               1995202696
account_id                                 524425745
ed_id                                              4
event_timestamp            2021-09-30 15:37:24+00:00
journey_steps_until_end                         2970
event_name_y                         browse_products
journey_id                                         1
milestone_number                                 NaN
stage                                 First Purchase
n_accounts                                      3277
Name: 54378282, dtype: object

In [19]:
# We want to study how the Discover phase affects success rates
# To do this, we first need to identify customers that have gone through this phase
discover_customers = merged_with_counts.groupby('customer_id')['stage'].apply(lambda x: 'Discover' in x.values).reset_index(name='has_discover')

merged_with_discover = pd.merge(merged_with_counts, discover_customers, on='customer_id')

In [20]:
# Save csv cuz my laptop slow af
merged_with_discover.to_csv('merged_with_discover.csv')

In [ ]:
# Delete these giant memory hogs
del merged, data, merged_with_counts

In [ ]:
merged_with_discover.head()

In [ ]:
merged_with_discover.describe()

In [ ]:
# We can repeat this for the 'First Purchase' stage to show which customers have made a purchase
first_purchase_customers = merged_with_counts.groupby('customer_id')['stage'].apply(lambda x: 'First Purchase' in x.values).reset_index(name='has_first_purchase')

merged_with_first_purchase = pd.merge(merged_with_counts, first_purchase_customers, on='customer_id')

In [18]:
merged_with_counts.stage.unique()

array(['First Purchase', 'Apply for Credit', 'Prospecting', 'Discover',
       'Downpayment', 'Order Shipped', 'Credit Account'], dtype=object)